# When Agents Disagree

In post 027, we added objects to the world. But what happens when **multiple agents want the same object**? Or when agents have **opposing goals**? Conflicts are inevitable in a multi-agent system.

This post explores:
- **Conflict Types**: Resource competition, goal conflicts, disagreements
- **Conflict Generation**: Creating scenarios where conflicts naturally arise
- **Resolution Strategies**: How agents negotiate, avoid, or escalate
- **Success Measurement**: Using ConvoKit metrics to evaluate resolution

## Conflict Scenarios

We'll test several conflict types:
1. **Resource Competition**: Multiple agents want the same object
2. **Goal Conflicts**: Agents have opposing objectives
3. **Disagreement**: Agents disagree on a course of action
4. **Territorial**: Agents want to occupy the same space


## Setup and Imports


In [ ]:
import os
import re
from dataclasses import dataclass, field
from typing import List, Optional, Dict
from enum import Enum
from dotenv import load_dotenv
from openai import OpenAI
import random

# ConvoKit imports
from convokit import Corpus, Utterance, Speaker
from convokit.text_processing import TextParser
from convokit.politeness_strategies import PolitenessStrategies
from convokit.coordination import Coordination
from collections import defaultdict
import numpy as np

load_dotenv("../../.env")
client = OpenAI()

GRID_SIZE = 20


## Conflict Types and Agent Goals


In [ ]:
class ConflictType(Enum):
    RESOURCE_COMPETITION = "resource_competition"
    GOAL_CONFLICT = "goal_conflict"
    DISAGREEMENT = "disagreement"
    TERRITORIAL = "territorial"

@dataclass
class AgentGoal:
    """A goal that an agent wants to achieve."""
    goal_type: str
    target: Optional[str] = None  # Object name, location, etc.
    priority: int = 5  # 1-10, higher = more important
    
@dataclass
class Agent:
    name: str
    x: int
    y: int
    color: str
    goals: List[AgentGoal] = field(default_factory=list)
    history: list = field(default_factory=list)
    
    def move(self, dx, dy):
        self.x = max(0, min(GRID_SIZE-1, self.x + dx))
        self.y = max(0, min(GRID_SIZE-1, self.y + dy))
    
    def get_position(self):
        return (self.x, self.y)
    
    def has_conflicting_goal_with(self, other: 'Agent') -> bool:
        """Check if this agent's goals conflict with another agent's goals."""
        for my_goal in self.goals:
            for their_goal in other.goals:
                # Resource competition: both want the same object
                if (my_goal.goal_type == "get_object" and 
                    their_goal.goal_type == "get_object" and
                    my_goal.target == their_goal.target):
                    return True
                # Goal conflict: opposing objectives
                if (my_goal.goal_type == "reach_location" and 
                    their_goal.goal_type == "prevent_location" and
                    my_goal.target == their_goal.target):
                    return True
        return False


## Conflict Generation

We'll create scenarios that naturally generate conflicts.


In [ ]:
def create_resource_conflict(agents: List[Agent], object_name: str):
    """Create a conflict where multiple agents want the same object."""
    # Assign the same goal to multiple agents
    for agent in agents:
        agent.goals.append(AgentGoal("get_object", object_name, priority=8))
    return ConflictType.RESOURCE_COMPETITION

def create_goal_conflict(agents: List[Agent], location: tuple):
    """Create a conflict where agents have opposing goals."""
    # One agent wants to reach a location, another wants to prevent it
    if len(agents) >= 2:
        agents[0].goals.append(AgentGoal("reach_location", str(location), priority=7))
        agents[1].goals.append(AgentGoal("prevent_location", str(location), priority=7))
    return ConflictType.GOAL_CONFLICT

def create_territorial_conflict(agents: List[Agent], location: tuple):
    """Create a conflict where agents want the same space."""
    for agent in agents:
        agent.goals.append(AgentGoal("occupy_location", str(location), priority=6))
    return ConflictType.TERRITORIAL

@dataclass
class Conflict:
    """Represents an active conflict."""
    conflict_type: ConflictType
    involved_agents: List[str]
    target: str
    resolved: bool = False
    resolution_strategy: Optional[str] = None


## Conflict Resolution Simulation

Agents will attempt to resolve conflicts through conversation and negotiation.


In [ ]:
def run_conflict_simulation(
    agents: List[Agent],
    conflict: Conflict,
    num_rounds: int = 15
) -> tuple[List[str], Conflict]:
    """Run a simulation with an active conflict."""
    transcript = []
    
    # Inform agents about the conflict
    conflict_description = ""
    if conflict.conflict_type == ConflictType.RESOURCE_COMPETITION:
        conflict_description = f"IMPORTANT: You and others want the {conflict.target}. This is a limited resource."
    elif conflict.conflict_type == ConflictType.GOAL_CONFLICT:
        conflict_description = f"IMPORTANT: You have opposing goals regarding {conflict.target}."
    elif conflict.conflict_type == ConflictType.TERRITORIAL:
        conflict_description = f"IMPORTANT: Multiple agents want to occupy {conflict.target}."
    
    for round_num in range(num_rounds):
        shuffled_agents = agents.copy()
        random.shuffle(shuffled_agents)
        
        for agent in shuffled_agents:
            if agent.name not in conflict.involved_agents:
                continue  # Only involved agents act in conflict resolution
            
            others = [a for a in agents if a != agent]
            others_loc = "\\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
            
            # Build goal description
            goals_desc = "\\nYour current goals:"
            for goal in agent.goals:
                goals_desc += f"\\n- {goal.goal_type}: {goal.target} (priority {goal.priority})"
            
            system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}
{goals_desc}

{conflict_description}

Recent chat messages:
{chr(10).join(transcript[-3:]) if transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - including negotiating, compromising, or discussing the conflict

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences). Try to resolve the conflict through conversation.
"""
            
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "system", "content": system_prompt}]
            )
            content = response.choices[0].message.content.strip()
            
            # Parse movement
            match = re.search(r'\\[MOVE:\\s*(\\w+)\\]', content)
            if match:
                direction = match.group(1).upper()
                if direction == "UP": agent.move(0, 1)
                elif direction == "DOWN": agent.move(0, -1)
                elif direction == "LEFT": agent.move(-1, 0)
                elif direction == "RIGHT": agent.move(1, 0)
            
            # Extract message
            message = re.sub(r'\\[MOVE:\\s*\\w+\\]', '', content).strip()
            if message:
                transcript.append(f"{agent.name}: {message}")
            
            # Check for resolution indicators
            message_lower = message.lower()
            resolution_keywords = ["agree", "compromise", "share", "take turns", "okay", "fine", "deal"]
            if any(keyword in message_lower for keyword in resolution_keywords):
                conflict.resolved = True
                conflict.resolution_strategy = "negotiation"
        
        # Check if conflict is resolved
        if conflict.resolved:
            break
    
    return transcript, conflict


In [ ]:
# Test 1: Resource Competition
print("=== Test 1: Resource Competition ===")
agents_resource = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

conflict_resource = Conflict(
    conflict_type=ConflictType.RESOURCE_COMPETITION,
    involved_agents=["Alice", "Bob"],
    target="key"
)

create_resource_conflict(agents_resource, "key")
transcript_resource, resolved_resource = run_conflict_simulation(agents_resource, conflict_resource)

print(f"\\nConflict resolved: {resolved_resource.resolved}")
print(f"Resolution strategy: {resolved_resource.resolution_strategy}")
print("\\nTranscript:")
for msg in transcript_resource:
    print(msg)

# Test 2: Goal Conflict
print("\\n\\n=== Test 2: Goal Conflict ===")
agents_goal = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

conflict_goal = Conflict(
    conflict_type=ConflictType.GOAL_CONFLICT,
    involved_agents=["Alice", "Bob"],
    target="(10, 10)"
)

create_goal_conflict(agents_goal, (10, 10))
transcript_goal, resolved_goal = run_conflict_simulation(agents_goal, conflict_goal)

print(f"\\nConflict resolved: {resolved_goal.resolved}")
print(f"Resolution strategy: {resolved_goal.resolution_strategy}")
print("\\nTranscript:")
for msg in transcript_goal:
    print(msg)


## Analyzing Conflict Resolution with ConvoKit

Let's measure how conflicts affect conversation dynamics.


In [ ]:
def conversation_to_corpus(transcript: List[str]) -> Corpus:
    """Convert transcript to ConvoKit Corpus."""
    utterances = []
    
    for idx, msg in enumerate(transcript):
        if ":" in msg:
            speaker_name, text = msg.split(":", 1)
            speaker_name = speaker_name.strip()
            text = text.strip()
            
            utterance = Utterance(
                id=f"utt_{idx}",
                speaker=Speaker(id=speaker_name),
                text=text
            )
            utterance.meta["timestamp"] = idx
            utterances.append(utterance)
    
    return Corpus(utterances=utterances)

def compute_dynamic_score(corpus: Corpus) -> float:
    """Dynamic: Collaborative (1) vs. Competitive (10)"""
    try:
        parser = TextParser()
        text_corpus = parser.transform(corpus)
        ps = PolitenessStrategies()
        ps_corpus = ps.transform(text_corpus)
        
        politeness_scores = []
        for utt in ps_corpus.iter_utterances():
            ps_score = utt.meta.get("politeness_strategies", {})
            positive_markers = sum([
                ps_score.get("feature_politeness_==HASPOSITIVE==", 0),
                ps_score.get("feature_politeness_==HASNEGATIVE==", 0) * -1,
            ])
            politeness_scores.append(positive_markers)
        
        avg_politeness = np.mean(politeness_scores) if politeness_scores else 0
        avg_politeness_normalized = min(1.0, max(0.0, avg_politeness / 5.0))
        
        speaker_counts = defaultdict(int)
        for utt in corpus.iter_utterances():
            speaker_counts[utt.speaker.id] += 1
        
        if len(speaker_counts) == 0:
            return 5.0
        
        total = sum(speaker_counts.values())
        probs = [count / total for count in speaker_counts.values()]
        entropy = -sum(p * np.log2(p) for p in probs if p > 0)
        max_entropy = np.log2(len(speaker_counts))
        balance_score = entropy / max_entropy if max_entropy > 0 else 0
        
        combined = (avg_politeness_normalized + balance_score) / 2
        score = 10 - (combined * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing dynamic score: {e}")
        return 5.0

def compute_conclusiveness_score(corpus: Corpus) -> float:
    """Conclusiveness: Consensus (1) vs. Divergence (10)"""
    try:
        coord = Coordination()
        coord_corpus = coord.fit_transform(corpus)
        
        agreement_markers = ["agree", "yes", "exactly", "right", "true", "correct", "indeed", "absolutely", "definitely", "deal", "compromise"]
        disagreement_markers = ["disagree", "no", "but", "however", "although", "wrong", "incorrect", "dispute", "differ"]
        
        agreement_count = 0
        disagreement_count = 0
        
        for utt in coord_corpus.iter_utterances():
            text_lower = utt.text.lower()
            for marker in agreement_markers:
                if marker in text_lower:
                    agreement_count += 1
            for marker in disagreement_markers:
                if marker in text_lower:
                    disagreement_count += 1
        
        if agreement_count == 0 and disagreement_count == 0:
            return 5.0
        elif disagreement_count == 0:
            return 1.0
        elif agreement_count == 0:
            return 10.0
        else:
            agreement_ratio = agreement_count / disagreement_count
        
        if agreement_ratio >= 2:
            score = 1 + (1 / (agreement_ratio - 1 + 1)) * 4
        elif agreement_ratio <= 0.5:
            score = 10 - (agreement_ratio * 4)
        else:
            score = 5.0
        
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing conclusiveness score: {e}")
        return 5.0

# Analyze conflict conversations
print("\\n=== Conflict Resolution Metrics ===")

corpus_resource = conversation_to_corpus(transcript_resource)
metrics_resource = {
    "dynamic": compute_dynamic_score(corpus_resource),
    "conclusiveness": compute_conclusiveness_score(corpus_resource)
}

print(f"\\nResource Competition:")
print(f"  Dynamic: {metrics_resource['dynamic']:.2f}/10")
print(f"  Conclusiveness: {metrics_resource['conclusiveness']:.2f}/10")
print(f"  Resolved: {resolved_resource.resolved}")

corpus_goal = conversation_to_corpus(transcript_goal)
metrics_goal = {
    "dynamic": compute_dynamic_score(corpus_goal),
    "conclusiveness": compute_conclusiveness_score(corpus_goal)
}

print(f"\\nGoal Conflict:")
print(f"  Dynamic: {metrics_goal['dynamic']:.2f}/10")
print(f"  Conclusiveness: {metrics_goal['conclusiveness']:.2f}/10")
print(f"  Resolved: {resolved_goal.resolved}")
